In [1]:
#!pip3 install http://download.pytorch.org/whl/cpu/torch-1.0.0-cp36-cp36m-linux_x86_64.whl

!pip install fastai==1.0.61
!pip install torch==1.9.0+cu102 -f https://download.pytorch.org/whl/torch_stable.html

     |████████████████████████████████| 239 kB 4.3 MB/s            
  Preparing metadata (setup.py) ... - done
  Created wheel for nvidia-ml-py3: filename=nvidia_ml_py3-7.352.0-py3-none-any.whl size=19191 sha256=7147062114de37b29a84046b3368eb61e4c7adcd1cebd1222b85fb8737e8103f
  Stored in directory: /root/.cache/pip/wheels/df/99/da/c34f202dc8fd1dffd35e0ecf1a7d7f8374ca05fbcbaf974b83
Successfully built nvidia-ml-py3
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.1.1
    Uninstalling typing-extensions-4.1.1:
      Successfully uninstalled typing-extensions-4.1.1
  Attempting uninstall: fastai
    Found existing installation: fastai 2.5.3
    Uninstalling fastai-2.5.3:
      Successfully uninstalled fastai-2.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-io 0.21.0 requires tensorflow-io-gcs-filesystem

In [2]:
!pip install sentencepiece

In [3]:
from fastai.text import *
import numpy as np
#import demoji
import pandas as pd
from sklearn.model_selection import train_test_split
import pickle
import sentencepiece as spm
import re
import pdb
from sklearn.utils import shuffle
from nltk.corpus import words
from bs4 import BeautifulSoup
import nltk
nltk.download('words')
import nltk, string, re, spacy,unicodedata, random
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import ToktokTokenizer
import nltk, string, re, spacy,unicodedata, random
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score

[nltk_data] Downloading package words to /usr/share/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
import fastai, torch
fastai.__version__ , torch.__version__

('1.0.61', '1.9.0+cu102')

In [5]:
torch.cuda.set_device(0)

In [6]:
def random_seed(seed_value, use_cuda):
    np.random.seed(seed_value)  
    torch.manual_seed(seed_value)  
    random.seed(seed_value)
    if use_cuda:
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)  
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

In [7]:
random_seed(42, True)

In [8]:
# Train Data
df_train_a = pd.read_csv('../input/emotion-analysis-in-tamil/Emotion_Analysis_Task_A_train.csv', sep='\t', header=None)
df_train_b = pd.read_csv('../input/emotion-analysis-in-tamil/Emotion_Analysis_Task_B_train.tsv', sep='\t', header=None)

# Dev Data
df_dev_a = pd.read_csv('../input/emotion-analysis-in-tamil/Emotion_Analysis_Task_A_dev.csv', sep='\t', header=None)
df_dev_b = pd.read_csv('../input/emotion-analysis-in-tamil/Emotion_Analysis_Task_B_dev.tsv', sep='\t', header=None)

# Test Data
df_test_a = pd.read_csv('../input/emotion-analysis-in-tamil/Emotion_Analysis_Task_A_test.csv', sep='\t', header=None)
df_test_b = pd.read_csv('../input/emotion-analysis-in-tamil/Emotion_Analysis_Task_B_test.csv', sep='\t', header=None)

In [9]:
path = Path('./')

In [10]:
!pwd

/kaggle/working


In [11]:
cat2ind = {'None-of-the-above': 0,
 'Transphobic': 1,
 'Counter-speech': 2,
 'Misandry': 3,
 'Homophobia': 4,
 'Hope-Speech': 5,
 'Xenophobia': 6,
 'Misogyny': 7,
 'Not-Tamil': 8}

ind2cat = dict()
for i in cat2ind:
    ind2cat[cat2ind[i]] = i

def restructure_df(df):
    txts = df[1].tolist()
    labs = df[0].tolist()
    labs_new = []
    for i in labs:
        labs_new.append(ind2cat[i])
    return pd.DataFrame({0:labs_new, 1:txts})

def over_under_sample(df, shuffle = True):
    def sample(rows, req_len):
        return [rows[i%len(rows)] for i in range(req_len)]

    if shuffle:
        df.sample(frac=1).reset_index(drop=True)
    data_dict = {'Misandry': [], 'Misogyny': [], 'Xenophobia': [], 'None-of-the-above': [], 'Hope-Speech': [], 'Counter-speech': [], 'Transphobic': [], 'Homophobia': [], 'Not-Tamil': []}

    for i in range(len(df)):
        data_dict[df[0][i]].append(list(df.iloc[i]))
    
    req_len = len(df)//len(data_dict)

    for lab in data_dict:
        data_dict[lab] = sample(data_dict[lab], req_len)
    df_rows = []
    for lab in data_dict:
        df_rows += data_dict[lab]
    df_new = pd.DataFrame(df_rows, columns = list(df.columns))
    if shuffle:
        df_new = df_new.sample(frac=1).reset_index(drop=True)
    return df_new

In [12]:
# df_train.shape, df_valid.shape, df_test.shape

In [13]:
df_train = df_train_a
df_valid = df_dev_a
df_test = df_test_a

In [14]:
df_train

,0,1
0,Neutral,நாளைக்கு அரிசிக்கு இந்த நிலமை வந்தா 🙂
1,Anger,மானம் கேட்ட அன்புமணி
2,Neutral,தவறு இஸ்ரேல் இருக்காது இதை நான் கூறவில்லை ஹமாஸ...
3,Joy,கொங்கு நாட்டு சிங்கம் உன்மையும் நேர்மையும் உலை...
4,Neutral,இவர் யார்? ஒவ்வொரு வார்த்தையும் முன்னுக்கு பின...
...,...,...
14203,Trust,பெ மணியரசன் கூறுவதைஉணர்ந்து. செயலாற்றுவதேஇன்ற...
14204,Ambiguous,இன்னும் எத்தன நாள் வச்சி செய்வீங்க.
14205,Anticipation,அடுத்த ஏதோ தயார்பன்னிட்டான்
14206,Ambiguous,தமிழ் மற்றும் சமஸ்கிருதம்


In [15]:
df_train = shuffle(df_train, random_state = 10).reset_index(drop=True)
df_valid = shuffle(df_valid, random_state = 10).reset_index(drop=True)
df_test = shuffle(df_test, random_state = 10).reset_index(drop=True)

df_train[1] = df_train[1].apply(str)
df_valid[1] = df_valid[1].apply(str)
df_test[1] = df_test[1].apply(str)

In [16]:
def deEmojify(string):
    emoji_pattern = re.compile("["
                              u"\U0001F600-\U0001F64F"  # emoticons
                              u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                              u"\U0001F680-\U0001F6FF"  # transport & map symbols
                              u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                              u"\U00002500-\U00002BEF"  # chinese char
                              u"\U00002702-\U000027B0"
                              u"\U00002702-\U000027B0"
                              u"\U000024C2-\U0001F251"
                              u"\U0001f926-\U0001f937"
                              u"\U00010000-\U0010ffff"
                              u"\u2640-\u2642"
                              u"\u2600-\u2B55"
                              u"\u200d"
                              u"\u23cf"
                              u"\u23e9"
                              u"\u231a"
                              u"\ufe0f"  # dingbats
                              u"\u3030"
                              "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

def preprocess(text):
    text = deEmojify(text) #convert emojis to their defns in words, they might be useful
    text = re.sub(r'([\.\'\"\/\-\_\--])',' ', text) # remove punctuations , removes @USER / some abbreviatins
    to_remove_url = ('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
      '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    text = re.sub(to_remove_url,'',text)  # remove url patterns
    text = re.sub(" \d+", " ", text)
    text = text.replace(","," ")
    text = re.sub(r'(?:^| )\w(?:$| )', ' ', text).strip()
    punctuation='!!"$%&()*+-/:;<=>?[\\]^_{|}~.'
    text = ''.join(ch for ch in text if ch not in set(punctuation))
    # text = text.translate(str.maketrans('', '', string.punctuation))
    text = BeautifulSoup(text, 'html.parser').get_text()
    # Stopword Removing
    tokenizer = ToktokTokenizer()
    # convert sentence into token of words
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    text = ' '.join(ch for ch in tokens)
    return text 

def clean(df):
    df[1] = df[1].apply(lambda x: preprocess(x))

clean(df_train)
clean(df_valid)
clean(df_test)

In [17]:
df_train[df_train[0].isnull()].shape, df_valid[df_valid[0].isnull()].shape, df_test[df_test[0].isnull()].shape

((0, 2), (0, 2), (0, 2))

In [18]:
class TamilTokenizer(BaseTokenizer):
    def __init__(self, lang:str):
        self.lang = lang
        self.sp = spm.SentencePieceProcessor()
        self.sp.Load(str('../input/spm-sota-models/tamil_spm_8k.model'))
        
    def tokenizer(self, t:str) -> List[str]:
        return self.sp.EncodeAsPieces(t)

In [19]:
sp = spm.SentencePieceProcessor()
sp.Load(str('../input/spm-sota-models/tamil_spm_8k.model'))
itos = [sp.IdToPiece(int(i)) for i in range(8000)]

In [20]:
tamil_vocab = Vocab(itos)

In [21]:
tokenizer = Tokenizer(tok_func=TamilTokenizer, lang='ta')

In [22]:
label_cols = [0]

In [23]:
data_lm = TextLMDataBunch.from_df(path=path, train_df=df_train, valid_df=df_valid, test_df=df_test, tokenizer=tokenizer, vocab=tamil_vocab)

/opt/conda/lib/python3.7/site-packages/fastai/core.py:302: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(a, dtype=dtype, **kwargs)


In [24]:
data_lm.show_batch()

idx,text
0,▁நினைத்த ால் ▁நான் ▁பொறுப்ப ல்ல ▁ x x b os ▁தளபதி ▁அப்ப வே ▁ஷோ ப ணா ▁ரவி ▁பா த்தி மா ▁பா பு வ ▁ம வுண்ட் ▁ரோ ட்டு ல ▁ஓட ▁ஓட ▁வி ர ட்டி ▁ . . . ▁ . . . ▁ . . . ▁சா க சம் ▁செய்த வர் ▁அவர் ட்ட ▁போ யி ▁ . . . ▁ . . . ▁ . . . ▁ x x b
1,▁சீ மான ின் ▁முக த் ▁திரை ▁ கிழ ிய ▁கண்டு ▁ர சி த்து ▁ம கிழ ் ந்து ▁சி ரி ப்பு ▁வந்த த ▁ x x b os ▁நல்ல ▁சாரா யம் ▁இது ▁மட்டும் ▁தான் ▁ x x b os ▁50 ▁ஆண்டு ▁காலம் ▁தமிழக ▁இளைஞர் களை ▁குடி கார ர் களா க்கி ▁பல ▁லட்ச ம் ▁பெண் களை ▁வித வை கள் ▁ஆ க்கி ▁ x x b os ▁விஷ ▁கருத்தை ▁தூ வ ுகிற ாய்
2,b os ▁சீ மான ின் ▁தமிழ் ▁வளர்ச்சி ▁ x x b os ▁இந்த ▁டாக்டர் கள் ▁இல்லை ய ென்ற ால் ▁கு ரா னா ▁சரியாக ி விடும் ▁கு ரா னா ▁ வுக்கு ▁முதல் ▁எதிரி கள் ▁மருத்துவர் கள் தான் ▁பொய் ▁பிர சார ம் ▁செய்து ▁விட்டு ▁பணத்தை ▁சம்பா தி க்கிறார் கள் ▁அ தற்காக ▁அரசாங்க த்திற்கு ▁சா த க மாக ▁பே ட்டி ▁அளிக்க ிறார்கள் ▁மருத்துவர் களைப் ▁பொறுத்த ▁மட்ட ில் ▁கொ ரோ னா ▁என்பது ▁ஒரு ▁பணம்
3,லா மே ▁ x x b os ▁பட த்தை ▁பார்த்து தான் ▁நா ச மாக போ கு து ▁ச மூ தா யம் ▁ x x b os ▁என் ▁வாக்கு ▁வீ ண் போ க வில்லை ▁அரு மை ▁மு ▁ஸ்டா லின் ▁ x x b os ▁தூத்துக்குடி ▁கா ப்பர் ▁நிறுவனம் ▁திறக்க ▁அனுமதி ▁ x x b os ▁அரு மை ▁அ ய்யா ▁ஆனால் ▁கிராம ▁கிளை ▁க ழ ங்கள் ▁வரை ▁உ ங்கள் ▁கட்சி ▁நிர்வாக ிகள்
4,▁ஏ த்திரு ப்பான ு க ▁மக்கள் ▁செ த்தா ▁என்ன னு ▁ x x b os ▁தளபதி ▁வாழ் ▁ x x b os ▁ஜீ ரக ம் ▁த ண்ணி ர் ▁குடி க்க லா ம ▁ x x b os ▁அ டே ய் ▁தி முக ▁அடி வரு டி ▁தமிழில் ▁மொழி மாற்றம் ▁செய் கி றா யா ▁து மா ரா ▁கா ண்ட் ▁மா ரே கா ▁ x x b os ▁ஏ ன் ▁யா ▁இது ▁தான்


In [25]:
awd_lstm_config = awd_lstm_lm_config.copy()
awd_lstm_config['n_hid'] = 1150
learn = language_model_learner(data_lm, arch=AWD_LSTM, drop_mult=0.3, config=awd_lstm_config, pretrained=False)

In [26]:
learn.load('/kaggle/input/pretrained-sota-models/wikitalm_8k_447_third')

LanguageLearner(data=TextLMDataBunch;

Train: LabelList (14208 items)
x: LMTextList
▁ x x b os ▁டே ய் ▁மதுரை ▁முத்து ▁நீ வந்த ியா ▁கா மெ டி ▁ப ண்ணி யா ▁மூட ிட்டு ▁போ,▁ x x b os ▁ஸ்ரீ ▁ரு த் ரம் ▁தமிழில் ▁கிடைக்க ு மா ▁ஜ யா,▁ x x b os ▁உயர மா ▁இருந்த ▁ஒரு ▁வேலை ▁எ கி றி ▁எ கி றி ▁அடி ப்பா னோ ▁சண்டை ▁வந்த ா ▁அடி ப்பா ங்க ளா ▁அதன் ▁சொன்ன ேன் ▁நான் யா ரும் ▁த ப்பா ▁நினைத்த ால் ▁நான் ▁பொறுப்ப ல்ல,▁ x x b os ▁தளபதி ▁அப்ப வே ▁ஷோ ப ணா ▁ரவி ▁பா த்தி மா ▁பா பு வ ▁ம வுண்ட் ▁ரோ ட்டு ல ▁ஓட ▁ஓட ▁வி ர ட்டி ▁ . . . ▁ . . . ▁ . . . ▁சா க சம் ▁செய்த வர் ▁அவர் ட்ட ▁போ யி ▁ . . . ▁ . . . ▁ . . .,▁ x x b os ▁எச் ▁ராஜா ▁பிராமண ர் களை ▁இந்துக் கள் ▁என்று ▁சொல் கிறார் ▁மற்றவர் களை ▁என்ன வ ென்று ▁சொல் வார்
y: LMLabelList
,,,,
Path: .;

Valid: LabelList (3552 items)
x: LMTextList
▁ x x b os ▁கவிஞர் ▁வைர முத்து ▁ஆண்ட ாள் ▁என்ற ▁கதை யை ▁சு ட்டி ▁காட்டி ▁பேச ▁அவருக்கு ▁எதிராக ▁போராட ிய வர்கள்,▁ x x b os ▁சென் செ ச் ▁புள்ளி ▁என்றால் ▁என்ன,▁ x x b os ▁அம்மா ▁பிரா மா ▁மு கு ர் த்தா ம் ▁எ ப்பொ ல் து ▁ஏ ல்ல ம் ▁வார ு த

In [27]:
df_train = df_train.drop([0]).reset_index().drop(["index"], axis=1)
df_train[0].value_counts()

Neutral         4841
Joy             2134
Ambiguous       1689
Trust           1254
Disguist         910
Anger            833
Anticipation     828
Sadness          695
Love             675
Surprise         248
Fear             100
Name: 0, dtype: int64

In [28]:
df_valid[0].value_counts()

Neutral         1222
Joy              558
Ambiguous        437
Trust            272
Anticipation     213
Disguist         210
Sadness          191
Love             189
Anger            184
Surprise          53
Fear              23
Name: 0, dtype: int64

In [29]:
df_test[0].value_counts()

Neutral         1538
Joy              702
Ambiguous        500
Trust            377
Disguist         277
Anticipation     271
Anger            244
Sadness          241
Love             196
Surprise          61
Fear              33
Name: 0, dtype: int64

In [30]:
learn.freeze()

In [31]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.610970,4.359224,0.322744,00:09


In [32]:
learn.unfreeze()

In [33]:
learn.fit_one_cycle(5, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,4.258491,4.208259,0.335280,00:11
1,4.110225,4.073703,0.349107,00:11
2,3.949435,4.017366,0.354347,00:11
3,3.831594,4.000637,0.356614,00:11
4,3.771197,3.999417,0.357178,00:11


In [34]:
learn.predict('▁தனது ▁சொந்த',n_words=10)

'▁தனது ▁சொந்த ▁அனுபவ ங்களை ▁அடக்க ▁ஆ சி ர் வாதம் ▁பெற ுவார் ▁நி'

In [35]:
learn.save_encoder('fine_tuned_enc')

In [36]:
data_clas = TextClasDataBunch.from_df(path=path, train_df=df_train, valid_df=df_valid, test_df=df_test, tokenizer=tokenizer, vocab=tamil_vocab, label_cols=label_cols, bs=64)

/opt/conda/lib/python3.7/site-packages/fastai/core.py:302: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(a, dtype=dtype, **kwargs)


/opt/conda/lib/python3.7/site-packages/numpy/core/_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [37]:
data_clas.show_batch()

text,target
▁ x x b os ▁த வரான ▁தகவல் ▁தரும் ▁ப யில் வான் ▁அவர் களே ▁ம க்கல் ▁தி லக ம் ▁தன் னுடன் ▁நடிக்க ும் ▁நடிகர் கள் ▁தொடங்கி ▁தன் னுடன் ▁வேலை ▁பார்க்க ும் ▁அ னை ▁வரு க்கும் ▁தான் ▁உ ன் னும் ▁அதே ▁உ ன வை ▁அனைவரும் ▁உ ன்ன வே ன் டும் ▁யா ரும் ▁ப சியா ே டு ▁இருக்க க்கூடாது ▁என்று ▁நினைத்த ▁ஒரே ▁நடிகர் ▁இந்திய ▁சினிமா ▁வரலாற்ற ி ள் ▁ம க்கல் ▁தி லக,Ambiguous
▁ x x b os ▁உணர் வாளர் களே ▁லட்ச க்கண க்கான ▁ரூபாய் ▁மதிப்பு ள்ள ▁ஆடை களில் ▁வல ம் ▁தொடர் ▁வெளிநாட்டு ▁சுற்றுலா ப் ▁பயண ங்கள் ▁க ள்ள ப் ▁பணத்தை ▁கண்டு ▁பிடிக்க ிற ேன் ▁பேர் வழி ▁என்கிற ▁பெயரில் ▁பண ▁மதிப்ப ிழப்பு ▁நடவடிக்கை ▁காண ாமல் ▁போன ▁தமிழக ▁மீ ன வர் களை ▁பற்றி ன ▁அக்கறை யின் மை ▁நீ ட் ▁தேர்வு க்கு ▁தமிழக த்திற்கு ▁எந்த ▁ஒரு ▁சரியான ▁ப திலும் ▁சொல்ல ாத ▁நிலை ▁ஸ் வ ச்,Neutral
▁ x x b os ▁மு ஹ ம் மது வின் ▁தக ப்பன் ▁அப்துல் லா ▁ஹா பல் ▁என்ற ▁தேவ தை யின் ▁அருள ால் ▁பிறந்த வா ் ▁மு ஹ ம் மது ▁தா த்தா விற்கு ▁பல ▁மனைவி கள் ▁ஆனால் ▁குழந்தைகள் ▁குறைவு ▁அவருக்கு ▁நிறைய ▁குழந்தைகள் ▁இல்லை யே ▁என்ற ▁குறை ▁எனவே ▁ஒரு ▁குறி ▁சொல்ல ும் ▁பெண்ண ின் ▁உதவி யோடு ▁ஹா பே ல் ▁என்ற ▁தேவ தை ▁கோவில ுக்கு ச் ▁சென்று ▁தனக்கு ▁நிறைய ▁ஆண் குழந்தை ▁வேண்டும் ▁என்றும்,Neutral
▁ x x b os ▁தமிழ் நா ட் ல ▁கலவர ம் ▁எப்படி யா ச்சு ம் ▁நடத்த னும் னு ▁அப்ப னும் ▁ம களும் ▁என்ன ▁என்ன ▁ஆட்ட ம் ▁போ ட்டீ ங்க ▁எ வ னோ ▁ஒரு த்தன் ▁செ ஞ்ச ▁த ப்பு க்கு ▁ஒரு ▁சமூக த்து ▁மே லை யே ▁ப ழி போ ட்டு ▁கலவர த்தை ▁தூண்ட ▁கலவர த்தை ▁தூண்ட ▁கு தி யா ▁கு தி ச்ச து ▁பிறப்ப ால் ▁வேறு தாய் ▁வயிற்று ▁பிள்ளை யாக ▁இருந்தாலும்,Neutral
▁ x x b os ▁திரு ▁மோகன் ▁சி ▁லா ச ரசு ▁அவர் களே ▁இந்தியாவில் ▁கிரு ஸ்த வர்கள் ▁கண்ண ீர் விட்டு ▁பிரா த்த னை ▁ப ண்ண ிய தால் தான் ▁இந்தியாவில் ▁கொ ரோ ணா ▁பாதிப்பு ▁லே சா க போ ய் விட்டது ▁என்று ▁உ ங்களுக்கு ▁வெ ட் கம் ▁ மான ம் ▁சூ டு ▁சூ ர னை ▁இல்லை யா ▁அப்படி ▁என்றால் ▁அமெரிக்கா ▁இத்தாலி ▁ஸ்பெயின ் ▁கனடா ▁பிரிட்டிஷ் ▁போன்ற ▁நாடுகளில் ▁பிரா த்த னை ▁செய்ய வில்லை,Disguist


In [38]:
del awd_lstm_config['tie_weights']
del awd_lstm_config['out_bias']

In [39]:
learn = text_classifier_learner(data_clas, arch=AWD_LSTM, drop_mult=0.5, config=awd_lstm_config)

In [40]:
learn.load_encoder('fine_tuned_enc')

RNNLearner(data=TextClasDataBunch;

Train: LabelList (14207 items)
x: TextList
▁ x x b os ▁ஸ்ரீ ▁ரு த் ரம் ▁தமிழில் ▁கிடைக்க ு மா ▁ஜ யா,▁ x x b os ▁உயர மா ▁இருந்த ▁ஒரு ▁வேலை ▁எ கி றி ▁எ கி றி ▁அடி ப்பா னோ ▁சண்டை ▁வந்த ா ▁அடி ப்பா ங்க ளா ▁அதன் ▁சொன்ன ேன் ▁நான் யா ரும் ▁த ப்பா ▁நினைத்த ால் ▁நான் ▁பொறுப்ப ல்ல,▁ x x b os ▁தளபதி ▁அப்ப வே ▁ஷோ ப ணா ▁ரவி ▁பா த்தி மா ▁பா பு வ ▁ம வுண்ட் ▁ரோ ட்டு ல ▁ஓட ▁ஓட ▁வி ர ட்டி ▁ . . . ▁ . . . ▁ . . . ▁சா க சம் ▁செய்த வர் ▁அவர் ட்ட ▁போ யி ▁ . . . ▁ . . . ▁ . . .,▁ x x b os ▁எச் ▁ராஜா ▁பிராமண ர் களை ▁இந்துக் கள் ▁என்று ▁சொல் கிறார் ▁மற்றவர் களை ▁என்ன வ ென்று ▁சொல் வார்,▁ x x b os ▁மிக்க ▁நல்ல ▁மன ு ஷன்
y: CategoryList
Ambiguous,Neutral,Disguist,Ambiguous,Love
Path: .;

Valid: LabelList (3552 items)
x: TextList
▁ x x b os ▁கவிஞர் ▁வைர முத்து ▁ஆண்ட ாள் ▁என்ற ▁கதை யை ▁சு ட்டி ▁காட்டி ▁பேச ▁அவருக்கு ▁எதிராக ▁போராட ிய வர்கள்,▁ x x b os ▁சென் செ ச் ▁புள்ளி ▁என்றால் ▁என்ன,▁ x x b os ▁அம்மா ▁பிரா மா ▁மு கு ர் த்தா ம் ▁எ ப்பொ ல் து ▁ஏ ல்ல ம் ▁வார ு தம் மா,▁ x x b os 

In [41]:
learn.freeze()

In [42]:
learn.loss_func.func

CrossEntropyLoss()

In [43]:
mcc = MatthewsCorreff()

In [44]:
f1 = FBeta(beta = 1,average = 'weighted')

In [45]:
learn.metrics = [f1]

In [46]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,f_beta,time
0,1.791715,1.709403,0.362917,00:07


/opt/conda/lib/python3.7/site-packages/numpy/core/_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)


In [47]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,f_beta,time
0,1.692553,1.647849,0.389770,00:07


[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
/opt/conda/lib/python3.7/site-packages/numpy/core/_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe

In [48]:
learn.unfreeze()
learn.fit_one_cycle(8, 1e-3, callbacks=[callbacks.SaveModelCallback(learn, every='improvement', monitor='f_beta', name='final')])

epoch,train_loss,valid_loss,f_beta,time
0,1.528037,1.637669,0.396657,00:16
1,1.452689,1.646598,0.403189,00:17
2,1.392088,1.714030,0.407467,00:17
3,1.360888,1.747329,0.405890,00:15
4,1.381437,1.768055,0.409628,00:17
5,1.270728,1.798557,0.405663,00:17
6,1.197799,1.825987,0.406260,00:17
7,1.142805,1.820909,0.406044,00:15


[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
/opt/conda/lib/python3.7/site-packages/numpy/core/_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe

Better model found at epoch 0 with f_beta value: 0.39665737748146057.


[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
/opt/conda/lib/python3.7/site-packages/numpy/core/_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe

Better model found at epoch 1 with f_beta value: 0.403189480304718.


[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
/opt/conda/lib/python3.7/site-packages/numpy/core/_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe

Better model found at epoch 2 with f_beta value: 0.4074665307998657.


[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
/opt/conda/lib/python3.7/site-packages/numpy/core/_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe

Better model found at epoch 4 with f_beta value: 0.40962815284729004.


[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
/opt/conda/lib/python3.7/site-packages/numpy/core/_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe

In [49]:
from sklearn.metrics import accuracy_score, matthews_corrcoef
df_dict = {'query': list(df_test[1]), 'actual_label': list(df_test[0]), 'predicted_label': ['']*df_test.shape[0]}
all_nodes = list(set(df_train[0]))
for node in all_nodes:
    df_dict[node] = ['']*df_test.shape[0]

i2c = {}
for key, value in learn.data.c2i.items():
    i2c[value] = key

df_result = pd.DataFrame(df_dict)
preds = learn.get_preds(ds_type=DatasetType.Test, ordered=True)
for index, row in df_result.iterrows():
    for node in all_nodes:
        row[node] = preds[0][index][learn.data.c2i[node]].item()
    row['predicted_label'] = i2c[np.argmax(preds[0][index]).data.item()]
df_result.head()

[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)


,query,actual_label,predicted_label,Love,Neutral,Sadness,Anticipation,Ambiguous,Trust,Joy,Surprise,Disguist,Fear,Anger
0,எப்டியோ பாவ பட்ட மக்களை கொன்னா பெருமை உங்களை ச...,Joy,Sadness,0.047571,0.038768,0.58417,0.058264,0.001432,0.06503,0.043902,0.002978,0.0064,0.135392,0.016093
1,தமிழ்நாடு அரசு எந்த ஆண்டுல இருந்து கடனில் இருந...,Neutral,Neutral,0.000415,0.42771,0.005176,0.040529,0.42687,0.004516,0.001216,0.003839,0.020353,0.000164,0.069211
2,விக்கிரமராஜா நல்லா இருக்கு உன் நியாயம்,Disguist,Neutral,0.016306,0.439274,0.010721,0.005467,0.073208,0.008315,0.163505,0.009358,0.165435,0.000618,0.107793
3,இதை தமிழில் சொல்லாதீர்கள் கன்னடத்திலோ,Neutral,Ambiguous,0.000405,0.080122,0.002651,0.002829,0.869848,0.000908,0.001106,0.002747,0.012214,0.000274,0.026896
4,சட்ட மன்ற உறுப்பினர் பணிதான் இது இதற்கு ஏன் இவ...,Disguist,Neutral,0.000135,0.515577,0.044854,0.007152,0.253739,0.003245,0.001472,0.001104,0.063699,0.000392,0.108631


In [50]:
accuracy_score(df_result['actual_label'], df_result['predicted_label'])

0.4349099099099099

In [51]:
matthews_corrcoef(df_result['actual_label'], df_result['predicted_label'])

0.2764966770896043

In [52]:
report = classification_report(df_result['actual_label'], df_result['predicted_label'],output_dict=True)

In [53]:
pd.DataFrame(report).transpose()

,precision,recall,f1-score,support
Ambiguous,0.478682,0.494000,0.486220,500.00000
Anger,0.238342,0.188525,0.210526,244.00000
Anticipation,0.297414,0.254613,0.274354,271.00000
Disguist,0.140187,0.054152,0.078125,277.00000
Fear,0.375000,0.090909,0.146341,33.00000
Joy,0.553360,0.598291,0.574949,702.00000
Love,0.247525,0.127551,0.168350,196.00000
Neutral,0.451114,0.644993,0.530907,1538.00000
Sadness,0.368056,0.219917,0.275325,241.00000
Surprise,0.000000,0.000000,0.000000,61.00000


In [54]:
df_result.to_csv('ULMFiT_ACD_TaEn_CD.csv', index=False)

In [55]:
import os
os.chdir(r'../working')
from IPython.display import FileLink
FileLink(r'ULMFiT_ACD_TaEn_CD.csv')

/kaggle/working/ULMFiT_ACD_TaEn_CD.csv